In [1]:
import requests
import pandas as pd
import time

# Season Info

In [2]:
userid = 'enter_user'
season = 2022
league_id = 'enter_league_id'

# API

In [3]:
league = requests.get('https://api.sleeper.app/v1/league/'+league_id).json()
draft_id = league['draft_id']

teams = requests.get('https://api.sleeper.app/v1/league/'+league_id+'/users').json()

draft = requests.get('https://api.sleeper.app/v1/draft/'+draft_id+'/picks').json()

# Get Manger Names

In [4]:
team_names = pd.DataFrame(columns = ['User ID', 'Manager'])

for i in range(len(teams)):
    user_id = teams[i]['user_id']
    display_name = teams[i]['display_name']
    
    team_names = team_names.append({'User ID': user_id, 'Manager': display_name}, ignore_index=True)

# Get Draft Info

In [5]:
df = pd.DataFrame(columns = ['Round', 'Pick', 'Player ID', 'Player', 'Position', 'User ID'])

for i in range(len(draft)):
    round_no = draft[i]['round']
    pick_no = draft[i]['pick_no']
    player_id = draft[i]['player_id']
    name = str(draft[i]['metadata']['first_name']) + ' ' + str(draft[i]['metadata']['last_name'])
    pos = draft[i]['metadata']['position']
    owner = draft[i]['picked_by']
    
    df = df.append({'Round': round_no, 'Pick': pick_no, 'Player ID': player_id,
                   'Player': name, 'Position': pos, 'User ID': owner}, ignore_index=True)

In [6]:
df = df.merge(team_names, how='left', left_on = 'User ID', right_on = 'User ID')

# Load Stats

In [7]:
qb =  pd.read_csv('2022qbstats.csv')
rb =  pd.read_csv('2022rbstats.csv')
wr =  pd.read_csv('2022wrstats.csv')
te =  pd.read_csv('2022testats.csv')
k =  pd.read_csv('2022kstats.csv')
dst =  pd.read_csv('2022dststats.csv')

In [8]:
for i in range(len(qb)):
    qb.at[i,'Player'] = qb.at[i,'Player'].strip()
    
for i in range(len(rb)):
    rb.at[i,'Player'] = rb.at[i,'Player'].strip()
    
for i in range(len(wr)):
    wr.at[i,'Player'] = wr.at[i,'Player'].strip()
    
for i in range(len(te)):
    te.at[i,'Player'] = te.at[i,'Player'].strip()
    
for i in range(len(k)):
    k.at[i,'Player'] = k.at[i,'Player'].strip()
    
for i in range(len(dst)):
    dst.at[i,'Player'] = dst.at[i,'Player'].strip()

# Merge Stats

In [9]:
stats = qb.append(rb).append(wr).append(te).append(k).append(dst)

In [10]:
Full = df.merge(stats[['Player','G','FPTS','FPTS/G']], how='left',left_on='Player', right_on='Player')

# Create Tiers for analysis purposes

In [11]:
Full["Draft Tier"] =  Full['Round'].map({1: 1, 2:1, 
                                          3:2, 4:2, 
                                          5:3,6:3,7:3,8:3,
                                          9:4,10:4,11:4,12:4,13:4,14:5,15:5,16:5,17:5
                                         })

Full["QT Draft Tier"] =  Full['Round'].map({1: 1, 2:1, 
                                          3:1, 4:1, 
                                          5:1,6:2,7:2,8:2,
                                          9:2,10:2,11:3,12:3,13:3,14:4,15:4,16:4,17:4
                                         })

In [12]:
for i in range(len(Full)):
    if Full.at[i,'Position'] == 'QB':
        Full.at[i,'Tier Med'] = Full.at[i,'FPTS'] - Full.loc[(Full['QT Draft Tier'] == Full.at[i,'QT Draft Tier']) & (Full['Position'] == Full.at[i,'Position'])]['FPTS'].median()
    elif Full.at[i,'Position'] == 'TE':
        Full.at[i,'Tier Med'] = Full.at[i,'FPTS'] - Full.loc[(Full['QT Draft Tier'] == Full.at[i,'QT Draft Tier']) & (Full['Position'] == Full.at[i,'Position'])]['FPTS'].median()
    else:
        Full.at[i,'Tier Med'] = Full.at[i,'FPTS'] - Full.loc[(Full['Draft Tier'] == Full.at[i,'Draft Tier']) & (Full['Position'] == Full.at[i,'Position'])]['FPTS'].median()
        
    if  Full.at[i,'Tier Med'] < 0: 
        Full.at[i,'Tier Med'] = Full.at[i,'Tier Med']/(2**(Full.at[i,'Draft Tier']-1))

In [13]:
Full['Tier Med'] = round(Full['Tier Med'],1)

In [14]:
Full

,Round,Pick,Player ID,Player,Position,User ID,Manager,G,FPTS,FPTS/G,Draft Tier,QT Draft Tier,Tier Med
0,1,1,4034,Christian McCaffrey,RB,335941988127891456,eddiey13,17.0,356.4,21.0,1,1,113.2
1,1,2,3198,Derrick Henry,RB,332669724372533248,SHEV,16.0,302.8,18.9,1,1,59.6
2,1,3,4029,Dalvin Cook,RB,599675198978981888,Jever42,17.0,237.8,14.0,1,1,-5.4
3,1,4,4035,Alvin Kamara,RB,332907205785317376,dvann811,15.0,211.7,14.1,1,1,-31.5
4,1,5,3164,Ezekiel Elliott,RB,332409631219408896,jwb4,15.0,185.8,12.4,1,1,-57.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,16,156,7611,Rhamondre Stevenson,RB,332409631219408896,jwb4,17.0,249.1,14.7,5,4,117.2
156,16,157,5119,Jason Sanders,K,332907205785317376,dvann811,17.0,135.0,7.9,5,4,-0.3
157,16,158,2747,Jason Myers,K,599675198978981888,Jever42,17.0,159.0,9.4,5,4,18.5
158,16,159,4144,Jonnu Smith,TE,332669724372533248,SHEV,12.0,52.0,4.3,5,4,0.0


# Write .csv

In [26]:
from pathlib import Path  
filepath = Path(r'{}.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
Full.to_csv(filepath.format(league_id))